In [ ]:
### Unique Identifier Extraction

(currently notes)

Anecdata - folks don't really comply with the URN spec but will include URN-like things.

I don't have any plan for identifying things tagged as "cool" URIs but aren't full URLs. 


####URNs We Can't Identify

Using some regex that is not really functional:

```
^([a-z0-9.#]{0,}:([a-z0-9][a-z0-9.-]{0,31}):[a-z0-9A-Z_()+,-.:=@;$!*\'%/?#\[\]]+)
```

and that is expecting 3 parts.

Not a URN (not saying they should be, either):

 - nlm:nlmuid-67840340R-bk
 - geonode:combined_tasks
 - gov.noaa.ngdc.stp.indices:G00587
 - gov.noaa.nmfs.inport:18623

 
 
Things that are a URN according to the regex:

 - urn:nbn:nl:ui:13-01hb-3b (extracted from a rule set)
 - oai:easy.dans.knaw.nl:easy-dataset:53685 (coded as a url)
 - oai:www.mpi.nl:MPI110411
 - clarin.eu:cr1:p_1271859438204
 - urn:uuid:6426c02b-f2b1-4326-a767-2384c303faf3
 - oai:oai.datacite.org:1979342
 
 
 
 

####More generally things that are awkward

A piece of text prior to identifier extraction. 

St1958-03-31T00:00:00-111.155555685007http://services.azgs.az.gov/OnlineAccessMineFiles/M-R/MilewidePima15.pdfDocument

It's composed of a timestamp, a URL and a file type. Using the regex, we would extract more text than just the URL for the URL identifier (so it would keep the ".pdfDocument".

### About the Identifiers

There are a couple of layers of extraction. There's a set of unqualified paths (they don't need to be the full path) for known identifier locations. We can't assume that these will conform to a unique identifier specfication or structure but is being provided as some identifier by the publisher. We are taking that in good faith.


**What it extracts**

URLS: 

http://vmo.igpp.ucla.edu/data1/SAMBA/2004/Feb/S04043VLD1s.asc
https://workspace.nottingham.ac.uk/pages/metadata/viewxml.action?pageId=132225079
ftp://ftp.nmh.ac.uk/wdc/obsdata/1minval
gsiftp://vetsman.ucar.edu:2811/


Handles:

http://hdl.handle.net/10022/AC:P:449


DOIs:

http://dx.doi.org/10.7916/D85B019G
10.7916/D85B019G
doi:10.5284/1017355


Cool URIs:

(a cool URI is pretty broad here - alphanumerics between forward slashes and a couple of select punctuation chars)

samos/data/research/WTEP/2009/WTEP_20090509v30001.nc
/opendap/hyrax/TRMM_L2/TRMM_2A23/2008/087/2A23.20080327.59048.7.HDF.Z

URNs:

SC:MOD17A2.005:2042282974
urn:uuid:c813a42d-0385-4cf8-b415-3e7c3783afef


UUIDs:

0d0a763e-76af-4e24-a5fd-c3930de5bed8

MD5s: 

cbb52feebe70d4d3821727b8d5dda65c


Identifier from a defined xpath rule:

gov.noaa.ngdc.sem.goes_xrs_g00036
freezingHeightIndex
V237


In [1]:
%matplotlib inline
import pandas as pd
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from IPython.display import display
from IPython.display import Image

In [2]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))

In [4]:
# percent of responses containing at least one of a type
sql = """
select match_type,
    round(count(distinct response_id) / 608968. * 100., 2) as pct_of_all_responses
from unique_identifiers
group by match_type;
"""
df = pd.read_sql(sql, engine)
df

,match_type,pct_of_all_responses
0,cooluri,10.48
1,doi,4.72
2,hdl,2.66
3,md5,6.29
4,text,10.17
5,url,100.00
6,urn,13.11
7,uuid,8.74


So every response has at least one identifier - the source url from the harvest. 

In [ ]:
# responses with one identifier that is the harveset url.
sql = """
with j as (
	with i as 
	(
		select response_id, count(potential_identifier) as potentials
		from unique_identifiers
		group by response_id
	)
	select r.id, r.source_url, i.potentials, u.potential_identifier, u.match_type
	from responses r 
		join i on i.response_id = r.id
		join unique_identifiers u on u.response_id = r.id
	where i.potentials = 1 and u.match_type = 'url'
)
select j.id, j.source_url
from j
where j.source_url = j.potential_identifier;

"""

### Looking at DOIs

Let's have a poke around to see where we find DOIs in identified responses (so ISO, FGDC, OpenSearch, OGC, OAI-PMH, and THREDDS). 

In [5]:
# some wonky sql
# NOTE: this is for any host/protocol pair with more than 10 responses total
sql = """
with i as (
	select d.response_id, (e.value->'protocol')::text as ident
	from identities d, jsonb_array_elements(d.identity::jsonb) e
	where d.identity is not null
), j as 
(
	select r.id, r.host, trim(both '"' from i.ident) as protocol
	from responses r join i on i.response_id = r.id
), k as (
	select j.host, j.protocol, count(distinct j.id) as total
	from j 
	group by j.host, j.protocol
)

select j.protocol, j.host, count(distinct j.id) as count_w_doi,
	round(count(distinct j.id) / max(k.total)::numeric * 100., 2) as pct_w_doi,
	max(k.total) as total_responses
from j 
	inner join unique_identifiers u on u.response_id = j.id
	join k on k.host = j.host and k.protocol = j.protocol
where u.match_type = 'doi' and k.total > 10
group by j.protocol, j.host
order by j.host, j.protocol, pct_w_doi desc;
"""

df = pd.read_sql(sql, engine)
df

,protocol,host,count_w_doi,pct_w_doi,total_responses
0,FGDC,arcticlcc.org,1,1.56,64
1,FGDC,astropedia.astrogeology.usgs.gov,35,97.22,36
2,FGDC,catalog.data.gov,27,0.89,3043
3,ISO,catalog.data.gov,150,3.07,4890
4,ISO,data.noaa.gov,66,5.31,1244
5,ISO,data-search.nerc.ac.uk,104,15.12,688
6,FGDC,data.usgs.gov,51,5.82,877
7,FGDC,edg.epa.gov,3,2.27,132
8,OpenSearch,gcmd.gsfc.nasa.gov,1,0.34,295
9,OpenSearch,gcmd.nasa.gov,1,1.41,71
